In [1]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

BASE_DIR = "gs://t5finetuning" #@param { type: "string" }
if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
DATA_DIR = os.path.join(BASE_DIR, "dataset")
MODELS_DIR = os.path.join(BASE_DIR, "models-quranqa-mt5-XL-v4")
ON_CLOUD = True


if ON_CLOUD:
  print("Setting up GCS access...")
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "v2-8"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.enable_eager_execution()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

Installing dependencies...
     |████████████████████████████████| 153 kB 5.4 MB/s 
     |████████████████████████████████| 90 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 74.8 MB/s 
     |████████████████████████████████| 366 kB 74.2 MB/s 
     |████████████████████████████████| 4.9 MB 72.2 MB/s 
     |████████████████████████████████| 4.3 MB 69.3 MB/s 
     |████████████████████████████████| 286 kB 83.6 MB/s 
     |████████████████████████████████| 4.0 MB 72.3 MB/s 
     |████████████████████████████████| 6.5 MB 69.3 MB/s 
     |████████████████████████████████| 596 kB 72.3 MB/s 
     |████████████████████████████████| 895 kB 73.3 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 462 kB 81.2 MB/s 
     |████████████████████████████████| 86 kB 4.5 MB/s 
Setting up GCS access...
Running on TPU: grpc://10.60.164.106:8470
Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Clear the registries
task_registry_names = list(t5.data.TaskRegistry.names())
for n in task_registry_names:
  t5.data.TaskRegistry.remove(n)

mixture_registry_names = list(t5.data.MixtureRegistry.names())
for n in mixture_registry_names:
  t5.data.MixtureRegistry.remove(n)

DEFAULT_SPM_PATH = "gs://t5-data/vocabs/mc4.250000.100extra/sentencepiece.model"


DEFAULT_TEMPERATURE = 1.0 / 0.3
DEFAULT_MIX_RATE = functools.partial(
    t5.data.rate_num_examples, temperature=DEFAULT_TEMPERATURE)

DEFAULT_VOCAB = t5.data.SentencePieceVocabulary(DEFAULT_SPM_PATH)
DEFAULT_OUTPUT_FEATURES = {
    "inputs": t5.data.Feature(
        vocabulary=DEFAULT_VOCAB, add_eos=True, required=False),
    "targets": t5.data.Feature(
        vocabulary=DEFAULT_VOCAB, add_eos=True)
}

In [3]:
import gzip
import json

# Public directory of Natural Questions data on GCS.
#NQ_JSONL_DIR = "gs://natural_questions/v1.0-simplified/"
#NQ_SPLIT_FNAMES = {
#    "train": "simplified-nq-train.jsonl.gz",
#    "validation": "nq-dev-all.jsonl.gz"
#}
#nq_counts_path = os.path.join(DATA_DIR, "nq-counts.json")
quranqa_tsv_path = {
    "train": os.path.join(DATA_DIR, "train.tsv"),
    #"valid": os.path.join(DATA_DIR, "valid-java.TSV"),
    "test": os.path.join(DATA_DIR, "dev.tsv")
}
#javasummary_tsv_path = {
#    "train": os.path.join(DATA_DIR, "data_javasummary-train-conala-augmented1.TSV"),
#    #"valid": os.path.join(DATA_DIR, "valid-java.TSV"),
#    "test": os.path.join(DATA_DIR, "data_javasummary-test-conala-augmented1.TSV")
#}

#def nq_jsonl_to_tsv(in_fname, out_fname):
#
#  def extract_answer(tokens, span):
#    """Reconstruct answer from token span and remove extra spaces."""
#    start, end = span["start_token"], span["end_token"]  
#    ans = " ".join(tokens[start:end])
#    # Remove incorrect spacing around punctuation.
#    ans = ans.replace(" ,", ",").replace(" .", ".").replace(" %", "%")
#    ans = ans.replace(" - ", "-").replace(" : ", ":").replace(" / ", "/")
#    ans = ans.replace("( ", "(").replace(" )", ")")
#    ans = ans.replace("`` ", "\"").replace(" ''", "\"")
#    ans = ans.replace(" 's", "'s").replace("s ' ", "s' ")
#    return ans
#
#  count = 0
#  with tf.io.gfile.GFile(in_fname, "rb") as infile,\
#       tf.io.gfile.GFile(out_fname, "w") as outfile:
#    for line in gzip.open(infile):
#      ex = json.loads(line)
#      # Remove any examples with more than one answer.
#      if len(ex['annotations'][0]['short_answers']) != 1:
#        continue
#      # Questions in NQ do not include a question mark.
#      question = ex["question_text"] + "?"
#      answer_span = ex['annotations'][0]['short_answers'][0]
#      # Handle the two document formats in NQ (tokens or text).
#      if "document_tokens" in ex:
#        tokens = [t["token"] for t in ex["document_tokens"]]
#      elif "document_text" in ex:
#        tokens = ex["document_text"].split(" ")
#      answer = extract_answer(tokens, answer_span)
#      # Write this line as <question>\t<answer>
#      outfile.write("%s\t%s\n" % (question, answer))
#      count += 1
#      tf.logging.log_every_n(
#          tf.logging.INFO,
#          "Wrote %d examples to %s." % (count, out_fname),
#          1000)
#    return count
#
#if tf.io.gfile.exists(nq_counts_path):
#  # Used cached data and counts.
#  tf.logging.info("Loading NQ from cache.")
#  num_nq_examples = json.load(tf.io.gfile.GFile(nq_counts_path))
#else:
#  # Create TSVs and get counts.
#  tf.logging.info("Generating NQ TSVs.")
#  num_nq_examples = {}
#  for split, fname in NQ_SPLIT_FNAMES.items():
#    num_nq_examples[split] = nq_jsonl_to_tsv(
#        os.path.join(NQ_JSONL_DIR, fname), nq_tsv_path[split])
#  json.dump(num_nq_examples, tf.io.gfile.GFile(nq_counts_path, "w"))

In [4]:
def quranqa_dataset_fn(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(quranqa_tsv_path[split])
  # Split each "<question>\t<answer>" example into (question, answer) tuple.
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["", ""],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  # Map each tuple to a {"question": ... "answer": ...} dict.
  ds = ds.map(lambda *ex: dict(zip(["question", "answer"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(quranqa_dataset_fn("train").take(5)):
  print(ex)


print("A few raw test examples...")
for ex in tfds.as_numpy(quranqa_dataset_fn("test").take(5)):
  print(ex)

A few raw train examples...
{'question': b'question: \xd9\x84\xd9\x85\xd8\xa7\xd8\xb0\xd8\xa7 \xd8\xb3\xd9\x8a\xd9\x8f\xd8\xad\xd8\xa7\xd8\xb3\xd8\xa8 \xd9\x88\xd9\x8a\xd9\x8f\xd8\xb9\xd8\xb0\xd8\xa8 \xd8\xa7\xd9\x84\xd8\xb6\xd8\xa7\xd9\x84 \xd9\x8a\xd9\x88\xd9\x85 \xd8\xa7\xd9\x84\xd9\x82\xd9\x8a\xd8\xa7\xd9\x85\xd8\xa9 \xd8\xa7\xd9\x86 \xd9\x83\xd8\xa7\xd9\x86 ""\xd9\x85\xd9\x86 \xd9\x8a\xd8\xb6\xd9\x84\xd9\x84 \xd8\xa7\xd9\x84\xd9\x84\xd9\x87 \xd9\x81\xd9\x85\xd8\xa7 \xd9\x84\xd9\x87 \xd9\x85\xd9\x86 \xd9\x87\xd8\xa7\xd8\xaf"" \xd9\x83\xd9\x85\xd8\xa7 \xd9\x88\xd8\xb1\xd8\xaf \xd9\x85\xd9\x86 \xd9\x82\xd9\x88\xd9\x84\xd9\x87 \xd8\xaa\xd8\xb9\xd8\xa7\xd9\x84\xd9\x89 \xd9\x81\xd9\x8a \xd8\xa2\xd9\x8a\xd8\xa9 23 \xd9\x88 \xd8\xa2\xd9\x8a\xd8\xa9 36 \xd9\x85\xd9\x86 \xd8\xb3\xd9\x88\xd8\xb1\xd8\xa9 \xd8\xa7\xd9\x84\xd8\xb2\xd9\x85\xd8\xb1\xd8\x9f context: \xd9\x88\xd9\x85\xd9\x86 \xd8\xa7\xd9\x84\xd9\x86\xd8\xa7\xd8\xb3 \xd9\x85\xd9\x86 \xd9\x8a\xd9\x82\xd9\x88\xd9\x84 \xd8\xa2\xd9\x85\

In [5]:
def quranqa_preprocessor(ds):
  def normalize_text(text):
    """Lowercase and remove quotes from a TensorFlow string."""
    #text = tf.strings.lower(text)
    #text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
    #text = ' '.join(tf.string_split(text).values)
    return text

  def to_inputs_and_targets(ex):
    """Map {"question": ..., "answer": ...}->{"inputs": ..., "targets": ...}."""
    return {
        "inputs":
             tf.strings.join(
                 ["question: ", normalize_text(ex["question"])]),
        "targets": normalize_text(ex["answer"])
    }
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [6]:
t5.data.TaskRegistry.remove("quranqa_t5")
t5.data.TaskRegistry.add(
    "quranqa_t5",
    # Specify the task type.
    t5.data.Task,
    # Supply a function which returns a tf.data.Dataset.
    dataset_fn=quranqa_dataset_fn,
    splits=["train", "test"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[quranqa_preprocessor],
    # Lowercase targets before computing metrics.
    postprocess_fn=t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    output_features=DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    #num_input_examples=num_nq_examples
)


In [7]:
quranqa_task = t5.data.TaskRegistry.get("quranqa_t5")
ds_test = quranqa_task.get_dataset(split="test", sequence_length={"inputs": 150, "targets": 30})
print("A few preprocessed test examples...")
for ex in tfds.as_numpy(ds_test.take(10)):
  print(ex)

ds_train = quranqa_task.get_dataset(split="train", sequence_length={"inputs": 100, "targets": 100})
print("A few preprocessed train examples...")
for ex in tfds.as_numpy(ds_train.take(10)):
  print(ex)

A few preprocessed test examples...
{'inputs_pretokenized': b'question: question: \xd8\xb6\xd8\xaf \xd9\x85\xd9\x86 \xd9\x81\xd9\x8f\xd8\xb1\xd8\xb6 \xd8\xa7\xd9\x84\xd8\xac\xd9\x87\xd8\xa7\xd8\xaf\xd8\x9f context: \xd9\x8a\xd8\xa7 \xd8\xa3\xd9\x8a\xd9\x87\xd8\xa7 \xd8\xa7\xd9\x84\xd9\x86\xd8\xa8\xd9\x8a \xd8\xac\xd8\xa7\xd9\x87\xd8\xaf \xd8\xa7\xd9\x84\xd9\x83\xd9\x81\xd8\xa7\xd8\xb1 \xd9\x88\xd8\xa7\xd9\x84\xd9\x85\xd9\x86\xd8\xa7\xd9\x81\xd9\x82\xd9\x8a\xd9\x86 \xd9\x88\xd8\xa7\xd8\xba\xd9\x84\xd8\xb8 \xd8\xb9\xd9\x84\xd9\x8a\xd9\x87\xd9\x85 \xd9\x88\xd9\x85\xd8\xa3\xd9\x88\xd8\xa7\xd9\x87\xd9\x85 \xd8\xac\xd9\x87\xd9\x86\xd9\x85 \xd9\x88\xd8\xa8\xd8\xa6\xd8\xb3 \xd8\xa7\xd9\x84\xd9\x85\xd8\xb5\xd9\x8a\xd8\xb1. \xd9\x8a\xd8\xad\xd9\x84\xd9\x81\xd9\x88\xd9\x86 \xd8\xa8\xd8\xa7\xd9\x84\xd9\x84\xd9\x87 \xd9\x85\xd8\xa7 \xd9\x82\xd8\xa7\xd9\x84\xd9\x88\xd8\xa7 \xd9\x88\xd9\x84\xd9\x82\xd8\xaf \xd9\x82\xd8\xa7\xd9\x84\xd9\x88\xd8\xa7 \xd9\x83\xd9\x84\xd9\x85\xd8\xa9 \xd8\xa7\xd9\x84\xd9\

In [8]:
t5.data.MixtureRegistry.remove("t5_quranqa_all")
t5.data.MixtureRegistry.add(
    "t5_quranqa_all",
    ["quranqa_t5"],
     default_rate=1.0
)

In [9]:
MODEL_SIZE = "3B" #@param["small", "base", "large", "3B", "11B"]
# Public GCS path for mT5 pre-trained model checkpoints
BASE_PRETRAINED_DIR = "gs://t5-data/pretrained_models"
PRETRAINED_DIR = os.path.join(BASE_PRETRAINED_DIR, MODEL_SIZE)
MODEL_DIR = os.path.join(MODELS_DIR, MODEL_SIZE)

if ON_CLOUD and MODEL_SIZE == "3B":
  tf.logging.warning(
      "The `3B` model is too large to use with the 5GB GCS free tier. "
      "Make sure you have at least 25GB on GCS before continuing."
  )
elif ON_CLOUD and MODEL_SIZE == "11B":
  raise ValueError(
      "The `11B` parameter is too large to fine-tune on the `v2-8` TPU "
      "provided by Colab. Please comment out this Error if you're running "
      "on a larger TPU."
  )

# Set parallelism and batch size to fit on v2-8 TPU (if possible).
# Limit number of checkpoints to fit within 5GB (if possible).
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]


tf.io.gfile.makedirs(MODEL_DIR)
# The models from our paper are based on the Mesh Tensorflow Transformer.
# Sequence-length values defined in: multilingual_t5/gin/sequence_lengths/xnli.gin
from t5.models import mtf_model 
model = mtf_model.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 354, "targets": 150},  
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
    extra_gin_bindings="Bitransformer.decode.max_decode_length = 128"
)

In [15]:
# Finetuning
FINETUNE_STEPS = 15000 #@param {type: "integer"}

model.finetune(
    mixture_or_task_name="t5_quranqa_all",
    pretrained_model_dir=PRETRAINED_DIR,
    finetune_steps=FINETUNE_STEPS
)

INFO:root:system_path_file_exists:gs://t5-data/pretrained_models/mt5/xl/operative_config.gin
ERROR:root:Path not found: gs://t5-data/pretrained_models/mt5/xl/operative_config.gin
INFO:root:Skipping import of unknown module `t5.data.sentencepiece_vocabulary` (skip_unknown=True).


INFO:tensorflow:Using config: {'_model_dir': 'gs://t5finetuning/models-quranqa-mt5-XL-v4/xl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.105.141.42:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.105.141.42:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.105.141.42:8470', '_evaluation_master': 'grpc://10.105.141.42

In [17]:
#Evaluation using accuracy as metric
FINETUNE_STEPS = 1015000

model.batch_size = 8
model.eval(
    mixture_or_task_name="t5_quranqa_all",
    checkpoint_steps=FINETUNE_STEPS,
    split="test"
)

INFO:root:system_path_file_exists:gs://t5finetuning/models-quranqa-mt5-XL-v4/xl/operative_config.gin
ERROR:root:Path not found: gs://t5finetuning/models-quranqa-mt5-XL-v4/xl/operative_config.gin
INFO:absl:Adding task 'quranqa_t5' with predict metric_fn(s).
INFO:absl:Skipping packing/padding for 'quranqa_t5' since sequence length is None.
INFO:absl:Setting sequence lengths to {'inputs': 380, 'targets': 130}
INFO:absl:Evaluating checkpoint step: 1015000


INFO:tensorflow:Using config: {'_model_dir': 'gs://t5finetuning/models-quranqa-mt5-XL-v4/xl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.105.141.42:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.105.141.42:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.105.141.42:8470', '_evaluation_master': 'grpc://10.105.141.42

INFO:absl:Padding 'quranqa_t5' with sequence lengths: {'inputs': 380, 'targets': 130}


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[8] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[model=8] LayoutRules{('vocab', 'model'), ('experts', 'batch'), ('heads', 'model'), ('batch', 'batch'), ('d_ff', 'model'), ('ensemble', 'ensemble')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7f229c1b3650>
INF

INFO:absl:eval/quranqa_t5/accuracy at step 1015000: 83.594


In [19]:
import random
import seqio
def print_random_predictions(task_name, n=100):
  """Print n predictions from the validation split of a task."""
  # Grab the dataset for this task.
  ds = seqio.TaskRegistry.get(task_name).get_dataset(
      split="test",
      sequence_length={"inputs": 200, "targets": 100},
      shuffle=False)

  def _prediction_file_to_ckpt(path):
    """Extract the global step from a prediction filename."""
    return int(path.split("_")[-2])

  # Grab the paths of all logged predictions.
  prediction_files = tf.io.gfile.glob(
      os.path.join(
          MODEL_DIR,
          "test_eval/%s_*_predictions" % task_name))
  # Get most recent prediction file by sorting by their step.
  latest_prediction_file = sorted(
      prediction_files, key=_prediction_file_to_ckpt)[-1]

  # Collect (inputs, targets, prediction) from the dataset and predictions file
  results = []
  with tf.io.gfile.GFile(latest_prediction_file) as preds:
    for ex, pred in zip(tfds.as_numpy(ds), preds):
      results.append((tf.compat.as_text(ex["inputs_pretokenized"]),
                      tf.compat.as_text(ex["targets_pretokenized"]),
                      pred.strip()))

  print("<== Random predictions for %s using checkpoint %s ==>\n" %
        (task_name, 
         _prediction_file_to_ckpt(latest_prediction_file)))

  for inp, tgt, pred in random.choices(results, k=10):
    print("Input:", inp)
    print("Target:", tgt)
    print("Prediction:", pred)
    print("Counted as Correct?", tgt == pred)
    print()

print_random_predictions("quranqa_t5")


<== Random predictions for quranqa_t5 using checkpoint 1015000 ==>

Input: question: question: هل هناك إسلام بدون الحديث الشريف؟ context: يوم تجد كل نفس ما عملت من خير محضرا وما عملت من سوء تود لو أن بينها وبينه أمدا بعيدا ويحذركم الله نفسه والله رءوف بالعباد. قل إن كنتم تحبون الله فاتبعوني يحببكم الله ويغفر لكم ذنوبكم والله غفور رحيم. قل أطيعوا الله والرسول فإن تولوا فإن الله لا يحب الكافرين.</s>
Target: قل أطيعوا الله والرسول
Prediction: قل أطيعوا الله والرسول
Counted as Correct? True

Input: question: question: هل هناك إسلام بدون الحديث الشريف؟ context: من يطع الرسول فقد أطاع الله ومن تولى فما أرسلناك عليهم حفيظا. ويقولون طاعة فإذا برزوا من عندك بيت طائفة منهم غير الذي تقول والله يكتب ما يبيتون فأعرض عنهم وتوكل على الله وكفى بالله وكيلا. أفلا يتدبرون القرآن ولو كان من عند غير الله لوجدوا فيه اختلافا كثيرا. وإذا جاءهم أمر من الأمن أو الخوف أذاعوا به ولو ردوه إلى الرسول وإلى أولي الأمر منهم لعلمه الذين يستنبطونه منهم ولولا فضل الله عليكم ورحمته لاتبعتم الشيطان إلا قليلا. فقاتل في سبيل

In [21]:
# Inference on some samples
import t5.data.mixtures

question_1 = "question: \u0645\u0627 \u0647\u064A \u0645\u0635\u0627\u0631\u0641 \u0627\u0644\u0632\u0643\u0627\u0629\u061F context: \u0625\u0646\u0645\u0627 \u0627\u0644\u0635\u062F\u0642\u0627\u062A \u0644\u0644\u0641\u0642\u0631\u0627\u0621 \u0648\u0627\u0644\u0645\u0633\u0627\u0643\u064A\u0646 \u0648\u0627\u0644\u0639\u0627\u0645\u0644\u064A\u0646 \u0639\u0644\u064A\u0647\u0627 \u0648\u0627\u0644\u0645\u0624\u0644\u0641\u0629 \u0642\u0644\u0648\u0628\u0647\u0645 \u0648\u0641\u064A \u0627\u0644\u0631\u0642\u0627\u0628 \u0648\u0627\u0644\u063A\u0627\u0631\u0645\u064A\u0646 \u0648\u0641\u064A \u0633\u0628\u064A\u0644 \u0627\u0644\u0644\u0647 \u0648\u0627\u0628\u0646 \u0627\u0644\u0633\u0628\u064A\u0644 \u0641\u0631\u064A\u0636\u0629 \u0645\u0646 \u0627\u0644\u0644\u0647 \u0648\u0627\u0644\u0644\u0647 \u0639\u0644\u064A\u0645 \u062D\u0643\u064A\u0645. \u0648\u0645\u0646\u0647\u0645 \u0627\u0644\u0630\u064A\u0646 \u064A\u0624\u0630\u0648\u0646 \u0627\u0644\u0646\u0628\u064A \u0648\u064A\u0642\u0648\u0644\u0648\u0646 \u0647\u0648 \u0623\u0630\u0646 \u0642\u0644 \u0623\u0630\u0646 \u062E\u064A\u0631 \u0644\u0643\u0645 \u064A\u0624\u0645\u0646 \u0628\u0627\u0644\u0644\u0647 \u0648\u064A\u0624\u0645\u0646 \u0644\u0644\u0645\u0624\u0645\u0646\u064A\u0646 \u0648\u0631\u062D\u0645\u0629 \u0644\u0644\u0630\u064A\u0646 \u0622\u0645\u0646\u0648\u0627 \u0645\u0646\u0643\u0645 \u0648\u0627\u0644\u0630\u064A\u0646 \u064A\u0624\u0630\u0648\u0646 \u0631\u0633\u0648\u0644 \u0627\u0644\u0644\u0647 \u0644\u0647\u0645 \u0639\u0630\u0627\u0628 \u0623\u0644\u064A\u0645." #@param {type:"string"}
question_2 = "question: \u0647\u0644 \u0633\u064A\u062F\u0646\u0627 \u0645\u062D\u0645\u062F (\u0635) \u0647\u0648 \u0623\u0648\u0644 \u0627\u0644\u0645\u0633\u0644\u0645\u064A\u0646\u061F context: \u0642\u0644 \u0625\u0646\u0646\u064A \u0647\u062F\u0627\u0646\u064A \u0631\u0628\u064A \u0625\u0644\u0649 \u0635\u0631\u0627\u0637 \u0645\u0633\u062A\u0642\u064A\u0645 \u062F\u064A\u0646\u0627 \u0642\u064A\u0645\u0627 \u0645\u0644\u0629 \u0625\u0628\u0631\u0627\u0647\u064A\u0645 \u062D\u0646\u064A\u0641\u0627 \u0648\u0645\u0627 \u0643\u0627\u0646 \u0645\u0646 \u0627\u0644\u0645\u0634\u0631\u0643\u064A\u0646. \u0642\u0644 \u0625\u0646 \u0635\u0644\u0627\u062A\u064A \u0648\u0646\u0633\u0643\u064A \u0648\u0645\u062D\u064A\u0627\u064A \u0648\u0645\u0645\u0627\u062A\u064A \u0644\u0644\u0647 \u0631\u0628 \u0627\u0644\u0639\u0627\u0644\u0645\u064A\u0646. \u0644\u0627 \u0634\u0631\u064A\u0643 \u0644\u0647 \u0648\u0628\u0630\u0644\u0643 \u0623\u0645\u0631\u062A \u0648\u0623\u0646\u0627 \u0623\u0648\u0644 \u0627\u0644\u0645\u0633\u0644\u0645\u064A\u0646." #@param {type:"string"}

questions = [question_1, question_2]

now = time.time()
# Write out the supplied questions to text files.
predict_inputs_path = os.path.join(MODEL_DIR, "predict_inputs_%d.txt" % now)
predict_outputs_path = os.path.join(MODEL_DIR, "predict_outputs_%d.txt" % now)
# Manually apply preprocessing by prepending "triviaqa question:".
with tf.io.gfile.GFile(predict_inputs_path, "w") as f:
  for q in questions:
    f.write("question: %s\n" % q.lower())

# Ignore any logging so that we only see the model's answers to the questions.
with tf_verbosity_level('ERROR'):
  model.batch_size = 8  # Min size for small model on v2-8 with parallelism 1.
  model.predict(
      input_file=predict_inputs_path,
      output_file=predict_outputs_path,
      # Select the most probable output token at each step.
      temperature=0,
  )

# The output filename will have the checkpoint appended so we glob to get 
# the latest.
prediction_files = sorted(tf.io.gfile.glob(predict_outputs_path + "*"))
print("\nPredictions using checkpoint %s:\n" % prediction_files[-1].split("-")[-1])
with tf.io.gfile.GFile(prediction_files[-1]) as f:
  for q, a in zip(questions, f):
    if q:
      print("Q: " + q)
      print("A: " + a)
      print()

INFO:root:system_path_file_exists:gs://t5finetuning/models-quranqa-mt5-XL-v4/xl/operative_config.gin
ERROR:root:Path not found: gs://t5finetuning/models-quranqa-mt5-XL-v4/xl/operative_config.gin



Predictions using checkpoint 1015000:

Q: question: ما هي مصارف الزكاة؟ context: إنما الصدقات للفقراء والمساكين والعاملين عليها والمؤلفة قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل فريضة من الله والله عليم حكيم. ومنهم الذين يؤذون النبي ويقولون هو أذن قل أذن خير لكم يؤمن بالله ويؤمن للمؤمنين ورحمة للذين آمنوا منكم والذين يؤذون رسول الله لهم عذاب أليم.
A: للفقراء والمساكين والعاملين عليها والمؤلفة قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل


Q: question: هل سيدنا محمد (ص) هو أول المسلمين؟ context: قل إنني هداني ربي إلى صراط مستقيم دينا قيما ملة إبراهيم حنيفا وما كان من المشركين. قل إن صلاتي ونسكي ومحياي ومماتي لله رب العالمين. لا شريك له وبذلك أمرت وأنا أول المسلمين.
A: أنا أول المسلمين




In [22]:
#Exportin and Saving the best model
EXPORT_DIR = os.path.join(BASE_DIR, "export-v3-XL")

model.batch_size = 1 # make one prediction per call
saved_model_path = model.export(
    EXPORT_DIR,
    checkpoint_step=-1,  # use most recent
    beam_size=1,  # no beam search
    temperature=1.0,  # sample according to predicted distribution
)
print("Model saved to:", saved_model_path)

INFO:root:system_path_file_exists:gs://t5finetuning/models-quranqa-mt5-XL-v4/xl/operative_config.gin
ERROR:root:Path not found: gs://t5finetuning/models-quranqa-mt5-XL-v4/xl/operative_config.gin


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_less_equal/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/check_ops.py", line 999, in assert_less_equal
    np.less_equal, x, y, data, summarize, message, name)  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/check_ops.py", line 425, in _binary_assert
    return control_flow_ops.Assert(condition, data, summarize=summarize)  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/traceback_utils.py", line 155, in error_handler
    del filtered_tb  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py", line 1082, in op_dispatch_handler
    return dispatch_target(*args, **kwargs)  File "/usr/local/lib/python3.7/dist-packages/tensorflow/

In [13]:
import tensorflow as tf
#import tensorflow_text  # Required to run exported model.

def load_predict_fn(model_path):
  if tf.executing_eagerly():
    print("Loading SavedModel in eager mode.")
    imported = tf.saved_model.load(model_path, ["serve"])
    return lambda x: imported.signatures['serving_default'](tf.constant(x))['outputs'].numpy()
  else:
    print("Loading SavedModel in tf 1.x graph mode.")
    tf.compat.v1.reset_default_graph()
    sess = tf.compat.v1.Session()
    meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], model_path)
    signature_def = meta_graph_def.signature_def["serving_default"]
    return lambda x: sess.run(
        fetches=signature_def.outputs["outputs"].name, 
        feed_dict={signature_def.inputs["inputs"].name: x}
    )



In [14]:
!pip install tensorflow_text

In [15]:
import tensorflow_text
saved_model_path = "gs://t5finetuning/export-v3-XL/1648490225"
predict_fn = load_predict_fn(saved_model_path)

Loading SavedModel in tf 1.x graph mode.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from gs://t5finetuning/export-compKag-3B-v1/1649281523/variables/variables


In [16]:
def answer(question):
  return predict_fn([question])[0].decode('utf-8')


In [8]:
question1 = "question: ما هو جزاء من يقول إن لله ولد؟ context: ولا يحزنك قولهم إن العزة لله جميعا هو السميع العليم. ألا إن لله من في السماوات ومن في الأرض وما يتبع الذين يدعون من دون الله شركاء إن يتبعون إلا الظن وإن هم إلا يخرصون. هو الذي جعل لكم الليل لتسكنوا فيه والنهار مبصرا إن في ذلك لآيات لقوم يسمعون. قالوا اتخذ الله ولدا سبحانه هو الغني له ما في السماوات وما في الأرض إن عندكم من سلطان بهذا أتقولون على الله ما لا تعلمون. قل إن الذين يفترون على الله الكذب لا يفلحون. متاع في الدنيا ثم إلينا مرجعهم ثم نذيقهم العذاب الشديد بما كانوا يكفرون."
print(answer(question1))

إن الذين يفترون على الله الكذب لا يفلحون. متاع في الدنيا ثم إلينا مرجعهم ثم نذيقهم العذاب الشديد الشديد بما كانوا يكفرون


In [ ]:
question2 = "question: متى يحل الإسلام دم الشخص؟ context: فليقاتل في سبيل الله الذين يشرون الحياة الدنيا بالآخرة ومن يقاتل في سبيل الله فيقتل أو يغلب فسوف نؤتيه أجرا عظيما. وما لكم لا تقاتلون في سبيل الله والمستضعفين من الرجال والنساء والولدان الذين يقولون ربنا أخرجنا من هذه القرية الظالم أهلها واجعل لنا من لدنك وليا واجعل لنا من لدنك نصيرا. الذين آمنوا يقاتلون في سبيل الله والذين كفروا يقاتلون في سبيل الطاغوت فقاتلوا أولياء الشيطان إن كيد الشيطان كان ضعيفا."
print(answer(question2))

الذين آمنوا يقاتلون في سبيل الله


In [27]:
'''
input_path = "data/qrcd_v1.1_dev.jsonl" 
data = []
with open(input_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))
        '''

In [28]:
#len(data)

109

In [29]:
#data[0]

{'answers': [{'start_char': 13,
   'text': 'للفقراء والمساكين والعاملين عليها والمؤلفة قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل'}],
 'passage': 'إنما الصدقات للفقراء والمساكين والعاملين عليها والمؤلفة قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل فريضة من الله والله عليم حكيم. ومنهم الذين يؤذون النبي ويقولون هو أذن قل أذن خير لكم يؤمن بالله ويؤمن للمؤمنين ورحمة للذين آمنوا منكم والذين يؤذون رسول الله لهم عذاب أليم.',
 'pq_id': '9:60-61_316',
 'question': 'ما هي مصارف الزكاة؟',
 'surah': 9,
 'verses': '60-61'}

In [16]:
'''
input_test_path = "data/qrcd_v1.1_test_noAnswers.jsonl" 
testdata = []
with open(input_test_path, 'r', encoding='utf-8') as f:
    for line in f:
        testdata.append(json.loads(line.rstrip('\n|\r')))
'''

In [17]:
#len(testdata)

274

In [18]:
#testdata[0]

{'answers': [],
 'passage': 'ولقد آتينا لقمان الحكمة أن اشكر لله ومن يشكر فإنما يشكر لنفسه ومن كفر فإن الله غني حميد. وإذ قال لقمان لابنه وهو يعظه يا بني لا تشرك بالله إن الشرك لظلم عظيم. ووصينا الإنسان بوالديه حملته أمه وهنا على وهن وفصاله في عامين أن اشكر لي ولوالديك إلي المصير. وإن جاهداك على أن تشرك بي ما ليس لك به علم فلا تطعهما وصاحبهما في الدنيا معروفا واتبع سبيل من أناب إلي ثم إلي مرجعكم فأنبئكم بما كنتم تعملون. يا بني إنها إن تك مثقال حبة من خردل فتكن في صخرة أو في السماوات أو في الأرض يأت بها الله إن الله لطيف خبير. يا بني أقم الصلاة وأمر بالمعروف وانه عن المنكر واصبر على ما أصابك إن ذلك من عزم الأمور. ولا تصعر خدك للناس ولا تمش في الأرض مرحا إن الله لا يحب كل مختال فخور. واقصد في مشيك واغضض من صوتك إن أنكر الأصوات لصوت الحمير.',
 'pq_id': '31:12-19_135',
 'question': 'ما هي وصايا لقمان لابنه؟',
 'surah': 31,
 'verses': '12-19'}

In [30]:
'''
run = {}
for e in data:
  id = e["pq_id"]
  question = "question: "+e["question"] + " context: "+e["passage"]
  sample = []
  for i in range(1,6):
    answer_element={}
    answer_element["answer"] = answer(question)
    answer_element["rank"] = i
    if i == 1:
      answer_element["score"] = 0.9886813087043423
    elif i == 2:
      answer_element["score"] = 0.7286813087043423
    elif i == 3:
      answer_element["score"] = 0.6886813087043423
    elif i == 4:
      answer_element["score"] = 0.5486813087043423
    else:
      answer_element["score"] = 0.4186813087043423
    
    sample.append(answer_element)
  run[id] = sample
  
  '''


In [19]:
'''
run = {}
for e in testdata:
  id = e["pq_id"]
  question = "question: "+e["question"] + " context: "+e["passage"]
  sample = []
  for i in range(1,6):
    answer_element={}
    answer_element["answer"] = answer(question)
    answer_element["rank"] = i
    if i == 1:
      answer_element["score"] = 0.9886813087043423
    elif i == 2:
      answer_element["score"] = 0.7286813087043423
    elif i == 3:
      answer_element["score"] = 0.6886813087043423
    elif i == 4:
      answer_element["score"] = 0.5486813087043423
    else:
      answer_element["score"] = 0.4186813087043423
    
    sample.append(answer_element)
  run[id] = sample
  
  '''


In [20]:
#run

{'10:107-109_372': [{'answer': 'يا أيها الناس قد جاءكم الحق من ربكم فمن اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها',
   'rank': 1,
   'score': 0.9886813087043423},
  {'answer': 'يا أيها الناس قد جاءكم الحق من ربكم فمن اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها',
   'rank': 2,
   'score': 0.7286813087043423},
  {'answer': 'يا أيها الناس قد جاءكم الحق من ربكم فمن اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها',
   'rank': 3,
   'score': 0.6886813087043423},
  {'answer': 'يا أيها الناس قد جاءكم الحق من ربكم فمن اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها',
   'rank': 4,
   'score': 0.5486813087043423},
  {'answer': 'يا أيها الناس قد جاءكم الحق من ربكم فمن اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها',
   'rank': 5,
   'score': 0.4186813087043423}],
 '10:15-17_372': [{'answer': 'قال الذين لا يرجون لقاءنا ائت بقرآن غير هذا أو بدله قل ما يكون لي أن أبدله من تلقاء نفسي إن أتبع إلا ما يوحى',
   'rank': 1,
   'score': 0.9886813087043423},
  {'answer': 'قال الذين لا يرجون لقاءنا ائت

In [ ]:
#run

In [21]:
#len(run)

274

In [22]:
# generate json run for the official evaluation
'''
import json
json_object = json.dumps(run, ensure_ascii=False)
with open("LARSA22_run02.json", "w", encoding='utf-8') as outfile:
    outfile.write(json_object)
    '''

In [34]:
#!pip install farasapy==0.0.14

In [35]:
# official evaluation of our best model on the dev set
#!python quranqa22_eval.py --run_file LARSA22_run02.json --gold_answers_file qrcd_v1.1_dev.jsonl

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
100% 241M/241M [00:27<00:00, 8.67MiB/s]
[2022-03-28 21:36:17,815 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.980255285201436, "exact_match": 0.9724770642201835, "f1": 0.980255285201436}


In [ ]:
# official evaluation of one of our models on the dev set
#!python quranqa22_eval.py --run_file LARSA22_run01.json --gold_answers_file qrcd_v1.1_dev.jsonl

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
100% 241M/241M [00:21<00:00, 11.2MiB/s]
[2022-03-27 15:21:03,824 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.916885449829249, "exact_match": 0.7155963302752294, "f1": 0.916885449829249}
